# Setup verification
This code contains brief code which aims to ensure that the various components of the robot setup are working correctly, are connected and so on. 

In [1]:
import sys
sys.path.append("..")

## AL5D robot

### In Linux

* The power source should be plugged in and robot turned on
* This should create a serial device: /dev/ttyUSB0 (or sometimes USB1)

```sudo chown <your-account> /dev/ttyUSB0``` 

### In Windows


### In the end:

If everything is successful, this should wake up the robot, perform a short move, than turn it out

In [2]:
from robot.al5d_position_controller import PositionController
# Linux
rob = PositionController("/dev/ttyUSB0") # or USB1 

# Windows (can be other COM#)
# rob = AL5D_PositionController("COM4")


In [3]:
pos = rob.get_position()
pos.height = pos.height + 1
rob.move(pos)

Shut down the robot: can be also used to terminate if other code dies.

In [4]:
rob.stop_robot()

# The camera

```
pip install opencv-python
pip install opencv-contrib-python
```

Capture device 0 is usually the webcam, the other ones seem to be 2... 

In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(2)
while(True):
    # Capture frame-by-frame
    
    ret, frame = cap.read()
    if not ret:
        print("Capture frame returned False, exiting the loop")
        break
    # dim = (config["image_width"], config["image_height"])
    #dim = (100, 100)
    # fixme: probably we want to crop as well??? 
    #imgresized = cv2.resize(frame, dim)
    #print(frame.shape)
    # it turns out that the small things are actually 480x640
    imgresized = cv2.resize(frame, (640, 480))
    imgresized = frame
    # ahat = rvc.control(imgresized)[0]
    #print(f"a^ = {ahat}")
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    #cv2.imshow('frame',gray)
    
    # frame = inference(frame)
    # wait for key q to quit
    #cv2.imshow('Press q to exit', frame)
    cv2.imshow('Press q to exit', imgresized)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    print("Exiting the acquisition loop as a result of a q key")
    #    break
    ret = cv2.waitKey(1)
    if ret != -1:
        print(ret)
        ret2 = ret & 0xFF
        print(ret2)
        if ret2 == ord('q'):
            break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

113
113


In [13]:
DIM = (256, 256)


# create the capture devices
capture_devs = {}
# camera0 - webcam on the computer
# camera2 - right mounted
# camera3 - the free floating one
# camera4 - the center mounted one 
# cameras = [0, 2, 3, 4]
# cameras = [0, 2]
cameras = [2]
for i in cameras:
    cap = cv2.VideoCapture(i) 
    if cap is None or not cap.isOpened():
        print(f"Warning: unable to open video source: {i}")
    else:
        capture_devs[f"dev{i}"] = cap
        print(f"cap{i} works")

while True:
    images = []
    for index in capture_devs.keys():
        cap = capture_devs[index]
        ret, frame = cap.read()
        if ret:
            # FIXME: probably we want to crop as well??? 
            imgresized = cv2.resize(frame, DIM)
            images.append(imgresized)
    concatenated_image = cv2.hconcat(images)
    cv2.imshow('Press q to exit these', concatenated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting the acquisition loop as a result of a q key")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

cap2 works
Exiting the acquisition loop as a result of a q key


In [3]:
f"{12:05d}"

'00012'

# The game controller
* Making sure that the game controller is working
* The version which is working with the Voyee 360 is the approxeng.input 2.5
* There was some kind of version interaction problem with version 2.6, it is unclear whether this is specific to my combinations in the hf workspace

* https://approxeng.github.io/approxeng.input/commandline.html 

```
pip install approxeng.input-2.5
```

# These are the buttons that we can read out
['square']
['triangle']
['circle']
['cross']
['l1']
['r1']
['ls']
['rs']
['ls']
['rs']
['dleft']
['dright']
['dup']
['ddown']

In [5]:
from approxeng.input.selectbinder import ControllerResource, ControllerNotFoundError
import time

try:
    with ControllerResource() as joystick:
        print('Found a joystick and connected')
        print(joystick.controls)
        while joystick.connected:
            presses = joystick.check_presses()
            if len(presses.buttons) > 0:
                print(presses.names)
                print(joystick)
                print(joystick["dleft"]) # if held, returns the seconds since held
except ControllerNotFoundError as e:
    print("Controller not found")
    print(e)
print("Bye")

Found a joystick and connected
{'axes': ['l', 'lt', 'lx', 'ly', 'r', 'rt', 'rx', 'ry'], 'buttons': ['circle', 'cross', 'ddown', 'dleft', 'dright', 'dup', 'home', 'l1', 'ls', 'r1', 'rs', 'select', 'square', 'start', 'triangle']}
['square']
SteamController, axes=['Left Horizontal=0', 'Left Vertical=0', 'Right Horizontal=0', 'Right Vertical=0', 'Left Trigger=0', 'Right Trigger=0', 'D-pad Horizontal=0', 'D-pad Vertical=0'], buttons=<approxeng.input.Buttons object at 0x7abef44fb490>
None
['triangle']
SteamController, axes=['Left Horizontal=0', 'Left Vertical=0', 'Right Horizontal=0', 'Right Vertical=0', 'Left Trigger=0', 'Right Trigger=0', 'D-pad Horizontal=0', 'D-pad Vertical=0'], buttons=<approxeng.input.Buttons object at 0x7abef44fb490>
None
['circle']
SteamController, axes=['Left Horizontal=0', 'Left Vertical=0', 'Right Horizontal=0', 'Right Vertical=0', 'Left Trigger=0', 'Right Trigger=0', 'D-pad Horizontal=0', 'D-pad Vertical=0'], buttons=<approxeng.input.Buttons object at 0x7abef44fb

KeyboardInterrupt: 

## Controller debug
These cells debug the Xbox360 controller based controller

In [2]:
from robot.al5d_pulse_controller import PulseController

ctrl = PulseController(device = "/dev/ttyUSB0")
while True:
    val = ctrl.query_position(3) # this is the elbow
    print(f"Elbow position {val}")

Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position 0
Elbow position

In [1]:
from xbox360.controller_xbox_al5d import XBox_AL5D_Controller

from al5d.position_controller import AL5D_PositionController

rob = AL5D_PositionController("/dev/ttyUSB0") # or USB1 
ctrl = XBox_AL5D_Controller(rob)
ctrl.control()
print("Exited the controller loop")

Found a joystick and connected
{'axes': ['l', 'lt', 'lx', 'ly', 'r', 'rt', 'rx', 'ry'], 'buttons': ['circle', 'cross', 'ddown', 'dleft', 'dright', 'dup', 'home', 'l1', 'ls', 'r1', 'rs', 'select', 'square', 'start', 'triangle']}
['square']
0
Exited the controller loop


### Space for experimentation with code snippets

In [3]:
images = {1: "aa", 2: "dsfsd"}
list(images.values())

['aa', 'dsfsd']

# Experimentation with the robot self-perception

In [ ]:
rob = AL5D_PositionController("/dev/ttyUSB0") # or USB1 